# Patent Deduplication

In [1]:
# uncomment and run if mismo is not installed
%pip install -q mismo ibis-framework[duckdb] altair

zsh:1: no matches found: ibis-framework[duckdb]
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import altair as alt
import ibis
from ibis import _
from ibis.expr import types as ir

import mismo

ibis.options.interactive = True

We are going to dedupe the PATSTAT dataset. Every record represents a patent filed,
and our task is to determine which one came from the same inventor.
The end goal is to add a column called `inventor_id` to each patent.
This dataset contains a ground truth label, so we can evaluate how well we did.

`Mismo` contains this as an included dataset so it is easy to get started.
The returned dataset is a `Linkage` dataclass, which is a container
for two tables of records (Linkage.left and Linkage.right) and a table of the
links between them (Linkage.links):

In [32]:
patents = mismo.playdata.load_patents()
print(patents)
display(patents.left.limit(5))
display(patents.right.limit(5))  # The same table as left
display(patents.links.limit(5))

Linkage<left=2_379, right=2_379, links=293_785>


┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ record_id ┃ label_true ┃ name_true            ┃ name                         ┃ latitude ┃ longitude ┃ coauthors                                                                        ┃ classes                                        ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ uint32    │ uint32     │ string               │ string                       │ float64  │ float64   │ string                                                                           │ string                                         │
├───────────┼────────────┼──────────────────────┼──────────────────────────────┼──────────┼───────────┼──────────────────────────────────────────────────────────────────────────────────┼────────────────────────────────────────────────┤
│      2909 │     402600 │ AGILENT TECHNOLOGIES │ * AGILENT TECHNOLOGIES, INC. │     0.00 │  0.000000 │ KONINK PHILIPS ELECTRONICS N V**DAVID E  SNYDER**THOMAS D  LYSTER                │ A61N**A61B                                     │
│      3574 │     569309 │ AKZO NOBEL           │ * AKZO NOBEL N.V.            │     0.00 │  0.000000 │ TSJERK  HOEKSTRA**ANDRESS K  JOHNSON**TERESA MARIE  CHERON**ALBERTO  SLIKTA**JA… │ G01N**B01L**C11D**G02F**F16L                   │
│      3575 │     569309 │ AKZO NOBEL           │ * AKZO NOBEL NV              │     0.00 │  0.000000 │ WILLIAM JOHN ERNEST  PARR**HANS  OSKARSSON**MARTIN  HELLSTEN**KORNELIS  OVERKEM… │ C09K**F17D**B01F**C23F                         │
│      3779 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │    52.35 │  4.916667 │ GUENTER  KOCHSMEIER**ZBIGNIEW  WIEGOLASKI**EVAN JOHN  STANBURY**PETER GRANT  JE… │ G02B**G04G**H02G**G06F                         │
│      3780 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │    52.35 │  4.916667 │ ZILAN  MANFRED**JOSIANE  RAMOS**DUANE LYNN  MORTENSEN**CHRISTIAN  LE SERGENT     │ H03G**B05D**H04L**H04B**C03B**C03C**G02B**H01B │
└───────────┴────────────┴──────────────────────┴──────────────────────────────┴──────────┴───────────┴──────────────────────────────────────────────────────────────────────────────────┴────────────────────────────────────────────────┘

┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ record_id ┃ label_true ┃ name_true            ┃ name                         ┃ latitude ┃ longitude ┃ coauthors                                                                        ┃ classes                                        ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ uint32    │ uint32     │ string               │ string                       │ float64  │ float64   │ string                                                                           │ string                                         │
├───────────┼────────────┼──────────────────────┼──────────────────────────────┼──────────┼───────────┼──────────────────────────────────────────────────────────────────────────────────┼────────────────────────────────────────────────┤
│      2909 │     402600 │ AGILENT TECHNOLOGIES │ * AGILENT TECHNOLOGIES, INC. │     0.00 │  0.000000 │ KONINK PHILIPS ELECTRONICS N V**DAVID E  SNYDER**THOMAS D  LYSTER                │ A61N**A61B                                     │
│      3574 │     569309 │ AKZO NOBEL           │ * AKZO NOBEL N.V.            │     0.00 │  0.000000 │ TSJERK  HOEKSTRA**ANDRESS K  JOHNSON**TERESA MARIE  CHERON**ALBERTO  SLIKTA**JA… │ G01N**B01L**C11D**G02F**F16L                   │
│      3575 │     569309 │ AKZO NOBEL           │ * AKZO NOBEL NV              │     0.00 │  0.000000 │ WILLIAM JOHN ERNEST  PARR**HANS  OSKARSSON**MARTIN  HELLSTEN**KORNELIS  OVERKEM… │ C09K**F17D**B01F**C23F                         │
│      3779 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │    52.35 │  4.916667 │ GUENTER  KOCHSMEIER**ZBIGNIEW  WIEGOLASKI**EVAN JOHN  STANBURY**PETER GRANT  JE… │ G02B**G04G**H02G**G06F                         │
│      3780 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │    52.35 │  4.916667 │ ZILAN  MANFRED**JOSIANE  RAMOS**DUANE LYNN  MORTENSEN**CHRISTIAN  LE SERGENT     │ H03G**B05D**H04L**H04B**C03B**C03C**G02B**H01B │
└───────────┴────────────┴──────────────────────┴──────────────────────────────┴──────────┴───────────┴──────────────────────────────────────────────────────────────────────────────────┴────────────────────────────────────────────────┘

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ record_id_l ┃ label_true_l ┃ name_true_l ┃ name_l            ┃ latitude_l ┃ longitude_l ┃ coauthors_l                                                                      ┃ classes_l                                      ┃ record_id_r ┃ label_true_r ┃ name_true_r      ┃ name_r                    ┃ latitude_r ┃ longitude_r ┃ coauthors_r                                                                ┃ classes_r              ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ uint32      │ uint32       │ string      │ string            │ float64    │ float64     │ string                                                                           │ string                                         │ uint32      │ uint32       │ string           │ string                    │ float64    │ float64     │ string                                                                     │ string                 │
├─────────────┼──────────────┼─────────────┼───────────────────┼────────────┼─────────────┼──────────────────────────────────────────────────────────────────────────────────┼────────────────────────────────────────────────┼─────────────┼──────────────┼──────────────────┼───────────────────────────┼────────────┼─────────────┼────────────────────────────────────────────────────────────────────────────┼────────────────────────┤
│        3574 │       569309 │ AKZO NOBEL  │ * AKZO NOBEL N.V. │       0.00 │    0.000000 │ TSJERK  HOEKSTRA**ANDRESS K  JOHNSON**TERESA MARIE  CHERON**ALBERTO  SLIKTA**JA… │ G01N**B01L**C11D**G02F**F16L                   │      579029 │       569309 │ AKZO NOBEL       │ Akzo-Nobel, N.V.          │       0.00 │    0.000000 │ BUMA BURSI ROBERTA**DE GOOYER MARCEL EVERT**VAN DER LOUW JAAP**LEYSEN DIRK │ C07J**A61K**A61P       │
│        3575 │       569309 │ AKZO NOBEL  │ * AKZO NOBEL NV   │       0.00 │    0.000000 │ WILLIAM JOHN ERNEST  PARR**HANS  OSKARSSON**MARTIN  HELLSTEN**KORNELIS  OVERKEM… │ C09K**F17D**B01F**C23F                         │      579029 │       569309 │ AKZO NOBEL       │ Akzo-Nobel, N.V.          │       0.00 │    0.000000 │ BUMA BURSI ROBERTA**DE GOOYER MARCEL EVERT**VAN DER LOUW JAAP**LEYSEN DIRK │ C07J**A61K**A61P       │
│        3779 │       656303 │ ALCATEL     │ * ALCATEL N.V.    │      52.35 │    4.916667 │ GUENTER  KOCHSMEIER**ZBIGNIEW  WIEGOLASKI**EVAN JOHN  STANBURY**PETER GRANT  JE… │ G02B**G04G**H02G**G06F                         │    31916707 │       656303 │ ALCATEL DITE SOC │ Societe Dite Alcatel N.V. │      52.35 │    4.916667 │ DELPRAT MARC**LASSAUX JEAN**GUEZOU JEAN                                    │ G06F**H04L**H04M**H04W │
│        3780 │       656303 │ ALCATEL     │ * ALCATEL N.V.    │      52.35 │    4.916667 │ ZILAN  MANFRED**JOSIANE  RAMOS**DUANE LYNN  MORTENSEN**CHRISTIAN  LE SERGENT     │ H03G**B05D**H04L**H04B**C03B**C03C**G02B**H01B │    31916707 │       656303 │ ALCATEL DITE SOC │ Societe Dite Alcatel N.V. │      52.35 │    4.916667 │ DELPRAT MARC**LASSAUX JEAN**GUEZOU JEAN                                    │ G06F**H04L**H04M**H04W │
│        3782 │       656303 │ ALCATEL     │ * ALCATEL N.V.    │       0.00 │    0.000000 │ OLIVIER  AUDOUIN**MICHEL  SOTOM**JEAN MICHEL  G

A Linkage is nothing fancy! It's just those tables.
So you can create a derived Linkage quite easily:

In [34]:
mismo.Linkage(left=patents.left, right=patents.right, links=patents.links.head(5))

Linkage<left=2_379, right=2_379, links=5>

The left, right, and links properties are not vanilla ibis.Tables though!
They are wrappers that act exactly like normal ibis.Tables,
except they have a few additional abilities.

For example, Linkage.left and Linkage.right are both instances of mismo.LinkedTable,
and hold references to the other table and the links table. Using this, you can eg
get the values from linked records in the other table,
using a syntax similar to `ibis.Table.select()`:

In [40]:
patents.left.with_linked_values(
    ibis._.latitude.max().name("max_latitude_of_all_linked_records"),
    other_names_by_default_collected_into_array="name",
).select(
    "name",
    "max_latitude_of_all_linked_records",
    "other_names_by_default_collected_into_array",
)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ name                         ┃ max_latitude_of_all_linked_records ┃ other_names_by_default_collected_into_array                            ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ string                       │ float64                            │ array<string>                                                          │
├──────────────────────────────┼────────────────────────────────────┼────────────────────────────────────────────────────────────────────────┤
│ * AGILENT TECHNOLOGIES, INC. │                               NULL │ NULL                                                                   │
│ * AKZO NOBEL N.V.            │                               52.5 │ ['Akzo-Nobel, N.V.', 'AKZO-NOBEL N.V.', ... +164]                      │
│ * AKZO NOBEL NV              │                               52.5 │ ['Akzo-Nobel, N.V.', 'AKZO-NOBEL N.V.', ... +163]                      │
│ * ALCATEL N.V.               │                               52.5 │ ['Societe Dite Alcatel N.V.', 'ALKATEL N.V.', ... +78]                 │
│ * ALCATEL N.V.               │                               52.5 │ ['Societe Dite Alcatel N.V.', 'ALKATEL N.V.', ... +77]                 │
│ * ALCATEL N.V.               │                               52.5 │ ['Societe Dite Alcatel N.V.', 'ALKATEL N.V.', ... +76]                 │
│ * CANON EUROPA N.V           │                               52.5 │ ['CANON EUROPA NV NAAMLOOZE VENNOOTSCHAP', 'Canon Europa NV', ... +24] │
│ * CANON EUROPA N.V.          │                               52.5 │ ['CANON EUROPA NV NAAMLOOZE VENNOOTSCHAP', 'Canon Europa NV', ... +23] │
│ * CANON EUROPA NV            │                               52.5 │ ['CANON EUROPA NV NAAMLOOZE VENNOOTSCHAP', 'Canon Europa NV', ... +22] │
│ * DSM N.V.                   │                               52.5 │ ['Naamloze Vennootschap DSM', 'Koninklijke DSM N.V.', ... +89]         │
│ …                            │                                  … │ …                                                                      │
└──────────────────────────────┴────────────────────────────────────┴────────────────────────────────────────────────────────────────────────┘

You can also get an overview of how many links each record has.
The link_counts() function gives you a histogram, answering the question
"There are {n_records} that have {n_links}".
This returned ibis.Table is another wrapper with an additional method,
`.chart()`, useful for visualization:

In [45]:
counts = patents.left.link_counts()
display(counts.head())
counts.chart()

┏━━━━━━━━━┳━━━━━━━━━━━┓
┃ n_links ┃ n_records ┃
┡━━━━━━━━━╇━━━━━━━━━━━┩
│ int64   │ int64     │
├─────────┼───────────┤
│      16 │        22 │
│     509 │         1 │
│     459 │         1 │
│      55 │        10 │
│      83 │         8 │
└─────────┴───────────┘

alt.VConcatChart(...)

See the docs on `LinkedTable` and `LinksTable` for more info.

I also want to mention that all this "magic" that you see is not mandatory:
I have tried very hard to make all the APIs transparent so that you could
re-create all this functionality yourself.
There isn't some data locked behind some private interface.
It all should be public, and free for you to modify as you desire.

## Blocking

OK, now that we have a primer on the dataset we are working with,
lets actually try to recreate it.
In the real world, this is the whole point of mismo:
given a table of records, you need to find the table of
links between them.

Let's clean the records up a bit:
- clean up whitespace
- convert the `classes` column to actual arrays (they really represent sets).
  Each element in `classes` is a 4-character IPC technical code that is like a tag
  for the patent. Similar patents will have similar tags.
- convert `coauthors` into a sorted list of tokens.

Of course, this sort of cleaning only happens after a lot of exploring the
data manually. This isn't anything in particular to record linkage,
but this needs to happen for any sort of data analysis.

If you're not familiar with Ibis, this is useful to give you a sense
of what it's API is like. Mismo tries to just use Ibis's API whenever possible,
and doesn't re-invent the wheel.


In [ ]:
from mismo.text import norm_whitespace


def clean_names(names: ir.StringValue) -> ir.StringValue:
    names = norm_whitespace(names)
    names = names.upper()
    # Only want to keep letters, numbers, and spaces
    names = names.re_replace("[^0-9A-Z ]", "")
    # Some company names have "NV" or "BV" in them, which is a Dutch
    # classification of businesses sort of like LLC vs Corporation in the US.
    # These seem to just be noise and don't help us distinguish between
    # different companies, so we want to remove them.
    # However, sometimes the names are ill-formed or have other junk in them
    # like "philipsN V" or "philipsN.V."
    names = names.re_replace(r"B\s*V", "")
    names = names.re_replace(r"N\s*V", "")
    # Also remove other common stopwords that usually just add noise
    names = names.re_replace(r"\bCO\b", "")
    names = names.re_replace(r"\bCORP\b", "")
    names = names.re_replace(r"\bLTD\b", "")
    names = names.re_replace(r"\bINC\b", "")
    names = names.re_replace(r"\bLLC\b", "")
    names = names.re_replace(r"\bNL\b", "")
    # Now have to do whitespace fixup again
    names = norm_whitespace(names)
    return names


def clean_coauthors(s: ir.StringValue) -> ir.ArrayValue:
    """Turn a 'list' of coauthors into a sorted array of cleaned name tokens.

    The raw string looks like:
    "DAVID E  SNYDER**THOMAS D  LYSTER"
    With "**" as the separator. However, entire names are not always useful
    to differentiate between people: there are some records where a coauthor is
    listed as "ALKEMA DUURT PIETER WILLEM" and another as "ALKEMA DUURT".
    These are probably the same person, but if we compared entire names with each other
    that would be a false negative. So instead we turn this into a sorted array of
    name tokens, eg for the example above:
    ["DAVID", "E", "SNYDER", "THOMAS", "D", "LYSTER"].

    Except we also want to remove initials and other stopwords like "VAN"
    so scrub all tokens less than 3 characters long.
    """
    s = s.upper().nullif("NONE").replace("**", " ")
    s = norm_whitespace(s)
    return s.split(" ").unique().sort().filter(lambda x: x.length() >= 3)


def clean_classes(s: ir.StringValue) -> ir.ArrayValue:
    return s.upper().split("**").map(norm_whitespace).sort()


cleaned_names = clean_names(_.name)

records = patents.left
featured = records.select(
    "record_id",
    "label_true",
    "name_true",
    "name",
    name_cleaned=cleaned_names,
    name_tokens=cleaned_names.split(" ").map(norm_whitespace).sort(),
    latitude=_.latitude.nullif(0),
    longitude=_.longitude.nullif(0),
    coauthors=clean_coauthors(records.coauthors),
    classes=clean_classes(records.classes),
)
featured = featured.cache()
featured

┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ record_id ┃ label_true ┃ name_true            ┃ name                         ┃ name_cleaned         ┃ name_tokens                 ┃ latitude ┃ longitude ┃ coauthors                         ┃ classes                  ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ uint32    │ uint32     │ string               │ string                       │ string               │ array<string>               │ float64  │ float64   │ array<string>                     │ array<string>            │
├───────────┼────────────┼──────────────────────┼──────────────────────────────┼──────────────────────┼─────────────────────────────┼──────────┼───────────┼───────────────────────────────────┼──────────────────────────┤
│      2909 │     402600 │ AGILENT TECHNOLOGIES │ * AGILENT TECHNOLOGIES, INC. │ AGILENT TECHNOLOGIES │ ['AGILENT', 'TECHNOLOGIES'] │     NULL │      NULL │ ['DAVID', 'ELECTRONICS', ... +5]  │ ['A61B', 'A61N']         │
│      3574 │     569309 │ AKZO NOBEL           │ * AKZO NOBEL N.V.            │ AKZO NOBEL           │ ['AKZO', 'NOBEL']           │     NULL │      NULL │ ['ALBERT', 'ALBERTO', ... +31]    │ ['B01L', 'C11D', ... +3] │
│      3575 │     569309 │ AKZO NOBEL           │ * AKZO NOBEL NV              │ AKZO NOBEL           │ ['AKZO', 'NOBEL']           │     NULL │      NULL │ ['CHRISTINA', 'ERNEST', ... +11]  │ ['B01F', 'C09K', ... +2] │
│      3779 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │ ALCATEL              │ ['ALCATEL']                 │    52.35 │  4.916667 │ ['EVAN', 'GRANT', ... +8]         │ ['G02B', 'G04G', ... +2] │
│      3780 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │ ALCATEL              │ ['ALCATEL']                 │    52.35 │  4.916667 │ ['CHRISTIAN', 'DUANE', ... +7]    │ ['B05D', 'C03B', ... +6] │
│      3782 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │ ALCATEL              │ ['ALCATEL']                 │     NULL │      NULL │ ['AUDOUIN', 'GABRIAGUES', ... +4] │ ['H01S', 'H04B', ... +1] │
│     15041 │    4333661 │ CANON EUROPA         │ * CANON EUROPA N.V           │ CANON EUROPA         │ ['CANON', 'EUROPA']         │     NULL │      NULL │ ['CANON', 'CENT', ... +11]        │ ['G06F']                 │
│     15042 │    4333661 │ CANON EUROPA         │ * CANON EUROPA N.V.          │ CANON EUROPA         │ ['CANON', 'EUROPA']         │     NULL │      NULL │ ['ADAM', 'ALEXANDER', ... +5]     │ ['G01B', 'G06T']         │
│     15043 │    4333661 │ CANON EUROPA         │ * CANON EUROPA NV            │ CANON EUROPA         │ ['CANON', 'EUROPA']         │     NULL │      NULL │ ['BARRY', 'BROCARD', ... +24]     │ ['B41J', 'G01B', ... +8] │
│     25387 │    7650783 │ DSM                  │ * DSM N.V.                   │ DSM                  │ ['DSM']                     │     NULL │      NULL │ ['ALEXANDER', 'ANDELA', ... +39]  │ ['A01H', 'A23J', ... +9] │
│         … │          … │ …                    │ …                            │ …                    │ …                           │        … │         … │ …                                 │ …                        │
└───────────┴────────────┴──────────────────────┴──────────────────────────────┴──────────────────────┴─────────────────────────────┴──────────┴───────────┴───────────────────────────────────┴──────────────────────────┘

OK, now it's time to block! This is where we generate comparisons between records.
If we were naive and generated all possible comparisons from N record,
you would end up with N^2 comparisons. For our small dataset of ~2000 records
we would be able to get away with this, but for datasets much larger than this
it would be infeasible.

Let's start with the most common kind of Linker: a `KeyLinker`.
Let's make a KeyLinker that creates links whereever the first 3 letters
of the `name_cleaned` column match between the two tables.

But, we also need to watch out for the case of really common values.
For example, in this dataset there are 505 records with a value of "KON".
If we joined on this value, this would result in ~(505 * 505) / 2 ~= 130_000 pairs.
This will be too slow.
So, by adding the max_pairs=10_000 param, before joining we efficiently remove
any records that would generate more than 10_000 pairs,
but still keep the records with rare values.

In [69]:
name_linker = mismo.KeyLinker(_.name_cleaned[0:3].name("first3"), max_pairs=10_000)
pair_counts = name_linker.pair_counts(featured, featured)
display(pair_counts.head())
pair_counts.chart()

┏━━━━━━━━┳━━━━━━━┓
┃ first3 ┃ n     ┃
┡━━━━━━━━╇━━━━━━━┩
│ string │ int64 │
├────────┼───────┤
│ ALC    │  4095 │
│ HIT    │  3916 │
│ ASM    │  3003 │
│ SCH    │  1770 │
│ SAR    │  1485 │
└────────┴───────┘

alt.VConcatChart(...)

Now we can generate a linkage using this by just calling the linker:

In [64]:
name_linkage = name_linker(featured, featured)
name_linkage

Linkage<left=2_379, right=2_379, links=20_335>

Under the hood, this is implemented by generating a join condition in SQL.
We can manually take a look at what's going on here:

In [65]:
left, right = featured, featured.view()
# condition is an ibis BooleanColumn
condition = name_linker.__join_condition__(left, right)
joined = mismo.join(left, right, condition)
joined.to_sql()

```sql
SELECT
  "t2"."record_id",
  "t2"."label_true",
  "t2"."name_true",
  "t2"."name",
  "t2"."name_cleaned",
  "t2"."name_tokens",
  "t2"."latitude",
  "t2"."longitude",
  "t2"."coauthors",
  "t2"."classes",
  "t3"."record_id" AS "record_id_right",
  "t3"."label_true" AS "label_true_right",
  "t3"."name_true" AS "name_true_right",
  "t3"."name" AS "name_right",
  "t3"."name_cleaned" AS "name_cleaned_right",
  "t3"."name_tokens" AS "name_tokens_right",
  "t3"."latitude" AS "latitude_right",
  "t3"."longitude" AS "longitude_right",
  "t3"."coauthors" AS "coauthors_right",
  "t3"."classes" AS "classes_right"
FROM "temp"."main"."ibis_cached_svtmw4n4rzcarbzvjrvofcdii4" AS "t2"
INNER JOIN "temp"."main"."ibis_cached_svtmw4n4rzcarbzvjrvofcdii4" AS "t3"
  ON ARRAY_SLICE("t2"."name_cleaned", COALESCE(0 + 1, 1), COALESCE(3, -1)) = ARRAY_SLICE("t3"."name_cleaned", COALESCE(0 + 1, 1), COALESCE(3, -1))
  AND NOT (
    "t2"."record_id" IN (
      SELECT
        "t7"."record_id"
      FROM (
        SELECT
          "t6".*
          EXCLUDE ("__temp___rymsv7odynarvbpdai6idz33ci")
        FROM (
          SELECT
            *
          FROM (
            SELECT
              "t0"."record_id",
              "t0"."label_true",
              "t0"."name_true",
              "t0"."name",
              "t0"."name_cleaned",
              "t0"."name_tokens",
              "t0"."latitude",
              "t0"."longitude",
              "t0"."coauthors",
              "t0"."classes",
              CAST(CAST(HASH(
                {'first3': ARRAY_SLICE("t0"."name_cleaned", COALESCE(0 + 1, 1), COALESCE(3, -1))}
              ) AS BIT) AS BIGINT) AS "__temp___rymsv7odynarvbpdai6idz33ci"
            FROM "temp"."main"."ibis_cached_svtmw4n4rzcarbzvjrvofcdii4" AS "t0"
          ) AS "t5"
          WHERE
            "t5"."__temp___rymsv7odynarvbpdai6idz33ci" IN (
              SELECT
                "t1"."__temp___rymsv7odynarvbpdai6idz33ci"
              FROM "temp"."main"."ibis_cached_cmu4iwrpb5hc3orhm46qmvm6fa" AS "t1"
            )
        ) AS "t6"
      ) AS "t7"
    )
  )
```

In addition to offering these code utilities, mismo tries to offer
some prebuilt solutions to common problems.
For example, we have a CoordinateLinker that
uses a KeyLinker internally to join records wherever (latitude, longitude)
pairs are close (they don't need to be exact!).

In [71]:
coord_blocker = mismo.lib.geo.CoordinateLinker(
    distance_km=10,
    lat="latitude",
    lon="longitude",
    max_pairs=50_000,
)
# above, we showed a pair_counts(), but we can also look at the distribution of keys
# within the records table
coord_blocker.key_counts_left(featured).chart()

alt.VConcatChart(...)

Notice how the name_linker has the method
`.__join_condition__(left: ibis.Table, right: ibis.Table) -> ibis.ir.BooleanValue | bool`?
This means it follows the mismo.HasJoinCondition Protocol.

Anything that implements this method is able to be combined
into a single `OrLinker`, which efficiently combines them
(eg removes duplicate pairs that would be generated by multiple sub-linkers)
and gives you the links generated from any of the sub-linkers:

In [73]:
sub_linkers = {
    "name": name_linker,
    "coauthors": mismo.KeyLinker("coauthors"),
    "coords": coord_blocker,
}
# If you didn't care about the names of the rules, you could also just pass
# in the blockers in a list.
or_linker = mismo.OrLinker(sub_linkers)
linkage = or_linker(featured, featured.view())
linkage

Linkage<left=2_379, right=2_379, links=93_396>

We can inspect which blocking rules are most to blame for the generated pairs. 
If some rules generate a huge amount of comparisons, it might be worth
trying to make them more restrictive so we get better performance.
Or, if some blocking rules aren't generating any comparisons,
that might be an indication that we have a bug in there somewhere.

In [ ]:
or_linker.upset_chart(featured, featured.view())

alt.VConcatChart(...)

By blocking, we reduced the number of needed pairs by a large factor.
In larger datasets, and with better blocking rules, this would be even more!

In [74]:
n_comparisons = linkage.links.count().execute()
n_naive = mismo.n_naive_comparisons(featured)
reduction_ratio = n_comparisons / n_naive
n_naive, n_comparisons, reduction_ratio

(2828631, 93396, 0.03301809249774891)

OK, now that we have our candidate pairs generated, let's actually do the
comparing of pairs.

In Mismo, similar to blocking, we use a single `PComparer` protocol to do all
comparisons. A `PComparer` is simply a callable of the form `(Table, **kwargs) -> Table`
that takes a table of blocked record pairs and returns a modified version.
Typical things a `PComparer` might do include

- Dropping some comparisons that are definitely not matches.
- Adding a score column for how much some feature matches, eg how well
  the address matches.
- Combining these individual per-feature scores into an overall score for the pair.

Here, we use one of the most common methods using LevelComparers,
each of which represents a measurement of similarity based on some dimension
(eg "location", "name", "price", etc).
Each LevelComparer is composed of MatchLevels, which represent discrete levels of agreement
(eg "exactly", "within 100km", "one or both values null", etc)

In [15]:
from mismo.compare import LevelComparer, MatchLevel
from mismo.lib.geo import distance_km
from mismo.sets import jaccard


class NameMatchLevel(MatchLevel):
    EXACT = 0
    SHARE_1_TOKEN = 1
    SHARE_2_PLUS_TOKENS = 2
    ELSE = 3


name_comparison = LevelComparer(
    name="Name",
    levels=NameMatchLevel,
    cases=[
        (_.name_cleaned_l == _.name_cleaned_r, NameMatchLevel.EXACT),
        (
            _.name_tokens_l.intersect(_.name_tokens_r).length() == 1,
            NameMatchLevel.SHARE_1_TOKEN,
        ),
        (
            _.name_tokens_l.intersect(_.name_tokens_r).length() >= 2,
            NameMatchLevel.SHARE_2_PLUS_TOKENS,
        ),
        (True, NameMatchLevel.ELSE),
    ],
)


class ClassMatchLevel(MatchLevel):
    EXACT = 0
    SHARE_1_CLASS = 1
    SHARE_2_PLUS_CLASSES = 2
    ELSE = 3


classes_comparison = LevelComparer(
    name="Classes",
    levels=ClassMatchLevel,
    cases=[
        (_.classes_l == _.classes_r, ClassMatchLevel.EXACT),
        (
            _.classes_l.intersect(_.classes_r).length() == 1,
            ClassMatchLevel.SHARE_1_CLASS,
        ),
        (
            _.classes_l.intersect(_.classes_r).length() >= 2,
            ClassMatchLevel.SHARE_2_PLUS_CLASSES,
        ),
        (True, ClassMatchLevel.ELSE),
    ],
)


class CoauthorMatchLevel(MatchLevel):
    JACCARD_50 = 0
    JACCARD_25 = 1
    JACCARD_10 = 2
    JACCARD_02 = 3
    ELSE = 4


coauthors_comparison = LevelComparer(
    name="Coauthors",
    levels=CoauthorMatchLevel,
    cases=[
        (jaccard(_.coauthors_l, _.coauthors_r) >= 0.5, CoauthorMatchLevel.JACCARD_50),
        (jaccard(_.coauthors_l, _.coauthors_r) >= 0.25, CoauthorMatchLevel.JACCARD_25),
        (jaccard(_.coauthors_l, _.coauthors_r) >= 0.1, CoauthorMatchLevel.JACCARD_10),
        (jaccard(_.coauthors_l, _.coauthors_r) >= 0.02, CoauthorMatchLevel.JACCARD_02),
        (True, CoauthorMatchLevel.ELSE),
    ],
)


class CoordMatchLevel(MatchLevel):
    EXACT = 0
    WITHIN_10KM = 1
    WITHIN_100KM = 2
    BOTH_MISSING = 3
    ONE_MISSING = 4
    ELSE = 5


coords_comparison = LevelComparer(
    name="Coords",
    levels=CoordMatchLevel,
    cases=[
        (_.latitude_l == _.latitude_r, CoordMatchLevel.EXACT),
        (
            distance_km(
                lat1=_.latitude_l,
                lon1=_.longitude_l,
                lat2=_.latitude_r,
                lon2=_.longitude_r,
            )
            <= 10,
            CoordMatchLevel.WITHIN_10KM,
        ),
        (
            distance_km(
                lat1=_.latitude_l,
                lon1=_.longitude_l,
                lat2=_.latitude_r,
                lon2=_.longitude_r,
            )
            <= 100,
            CoordMatchLevel.WITHIN_100KM,
        ),
        (
            ibis.and_(
                _.latitude_l.isnull() | _.longitude_l.isnull(),
                _.latitude_r.isnull() | _.longitude_r.isnull(),
            ),
            CoordMatchLevel.BOTH_MISSING,
        ),
        (
            ibis.or_(
                _.latitude_l.isnull() | _.longitude_l.isnull(),
                _.latitude_r.isnull() | _.longitude_r.isnull(),
            ),
            CoordMatchLevel.ONE_MISSING,
        ),
        (True, CoordMatchLevel.ELSE),
    ],
)

comparers = [
    name_comparison,
    classes_comparison,
    coauthors_comparison,
    coords_comparison,
]

compared = linkage.links
for comp in comparers:
    compared = comp(compared)
compared = compared.cache()
compared

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┓
┃ record_id_l ┃ label_true_l ┃ name_true_l                      ┃ name_l                                            ┃ name_cleaned_l                             ┃ name_tokens_l                              ┃ latitude_l ┃ longitude_l ┃ coauthors_l                       ┃ classes_l                 ┃ record_id_r ┃ label_true_r ┃ name_true_r                  ┃ name_r                              ┃ name_cleaned_r                  ┃ name_tokens_r                   ┃ latitude_r ┃ longitude_r ┃ coauthors_r                   ┃ classes_r                 ┃ Name ┃ Classes ┃ Coauthors ┃ Coords ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━┩
│ uint32      │ uint32       │ string                           │ string                                            │ string                                     │ array<string>                              │ float64    │ float64     │ array<string>                     │ array<string>             │ uint32      │ uint32       │ string                       │ string                              │ string                          │ array<string>                   │ float64    │ float64     │ array<string>                 │ array<string>             │ int8 │ int8    │ int8      │ int8   │
├─────────────┼──────────────┼──────────────────────────────────┼───────────────────────────────────────────────────┼────────────────────────────────────────────┼────────────────────────────────────────────┼────────────┼─────────────┼───────────────────────────────────┼───────────────────────────┼─────────────┼──────────────┼──────────────────────────────┼─────────────────────────────────────┼─────────────────────────────────┼─────────────────────────────────┼────────────┼─────────────┼───────────────────────────────┼───────────────────────────┼──────┼─────────┼───────────┼────────┤
│        3779 │       656303 │ ALCATEL                          │ * ALCATEL N.V.                                    │ ALCATEL                                    │ ['ALCATEL']                                │  52.350000 │    4.916667 │ ['EVAN', 'GRANT', ... +8]         │ ['G02B', 'G04G', ... +2]  │    35414627 │     35444479 │ VAILLANT                     │ Vaillant B.V.                       │ VAILLANT                        │ ['VAILLANT']                    │  52.350000 │    4.916667 │ ['ACHIM', 'ANDREAS', ... +70] │ ['C23D', 'F23D', ... +10] │    3 │       1 │         4 │      0 │
│        3780 │       656303 │ ALCATEL                          │ * ALCATEL N.V.                                    │ ALCATEL                                    │ ['ALCATEL']                                │  52.350000 │    4.916667 │ ['CHRISTIAN', 'DUANE', ... +7]    │ ['B05D', 'C03B', ... +6]  │    35414627 │     35444479 │ VAILLANT                     │ Vaillant B.V.                       │ VAILLANT                        │ ['VAILLANT']                 

The result above is the blocked table, with a column added for every `Comparer`.
The value of each column is the level that the record pair matched at.
For example, there is now a "Name" column,
filled with values like "exact", "share 1 token", etc.

Now that we have our features, we can use the Fellegi-Sunter model to train weights
for each of these features. This is a probabilistic model that is based on the concept
of odds. When you see an exact match on name, that increases the odds of a match
by some amount, maybe 10x. When you see a non-match on name, that decreases the odds
of a match by some amount, maybe 0.1x. We can either train this from labeled data,
or we can use unlabeled data using an algorithm called "Expectation Maximization".

In [16]:
weights = mismo.fs.train_using_em(comparers, featured, featured, max_pairs=100_000)
# Can save and load weights
# weights.to_json("weights.json")
# weights = ComparisonWeights.from_json("weights.json")
weights.plot()

alt.VConcatChart(...)

In the above plot, you can see the odds for "share 1 class" and "share 2+ classes"
are nearly the same. That doesn't quite make sense, and might be an indicator
that we should look more into this to see if there is a problem with the features
that we are using. But for now let's just move on.

Use the weights to score the record pairs, finding the odds for each
`Comparer`, and then combining them into an overall odds for the record pair.

In [17]:
scored = weights.score_compared(compared)
scored = scored.cache()
scored

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━┓
┃ record_id_l ┃ label_true_l ┃ name_true_l                      ┃ name_l                                            ┃ name_cleaned_l                             ┃ name_tokens_l                              ┃ latitude_l ┃ longitude_l ┃ coauthors_l                       ┃ classes_l                 ┃ record_id_r ┃ label_true_r ┃ name_true_r                  ┃ name_r                              ┃ name_cleaned_r                  ┃ name_tokens_r                   ┃ latitude_r ┃ longitude_r ┃ coauthors_r                   ┃ classes_r                 ┃ odds         ┃ Name ┃ Name_odds ┃ Classes ┃ Classes_odds ┃ Coauthors ┃ Coauthors_odds ┃ Coords ┃ Coords_odds ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━┩
│ uint32      │ uint32       │ string                           │ string                                            │ string                                     │ array<string>                              │ float64    │ float64     │ array<string>                     │ array<string>             │ uint32      │ uint32       │ string                       │ string                              │ string                          │ array<string>                   │ float64    │ float64     │ array<string>                 │ array<string>             │ float64      │ int8 │ float64   │ int8    │ float64      │ int8      │ float64        │ int8   │ float64     │
├─────────────┼──────────────┼──────────────────────────────────┼───────────────────────────────────────────────────┼────────────────────────────────────────────┼────────────────────────────────────────────┼────────────┼─────────────┼───────────────────────────────────┼───────────────────────────┼─────────────┼──────────────┼──────────────────────────────┼─────────────────────────────────────┼─────────────────────────────────┼─────────────────────────────────┼────────────┼─────────────┼───────────────────────────────┼───────────────────────────┼──────────────┼──────┼───────────┼─────────┼──────────────┼───────────┼────────────────┼────────┼─────────────┤
│        3779 │       656303 │ ALCATEL                          │ * ALCATEL N.V.                                    │ ALCATEL                                    │ ['ALCATEL']                                │  52.350000 │    4.916667 │ ['EVAN', 'GRANT', ... +8]         │ ['G02B', 'G04G', ... +2]  │    35414627 │     35444479 │ VAILLANT                     │ Vaillant B.V.                       │ VAILLANT                        │ ['VAILLANT']                    │  52.350000 │    4.916667 │ ['ACHIM', 'ANDREAS', ... +70] │ ['C23D', 'F23D', ... +10] │ 4.195552e-03 │    3 │  0.000228 │       1 │     5.416808 │         4 │       0.831951 │      0 │    4.082939 │
│        3780 │       

We can plot these compared pairs.
We can see which comparison levels are most common,
which occur together,
which lead to matches, and which lead to non-matches.

The exact match levels have the highest odds, and the
else levels have the lowest. The other levels are somewhere in between.

In [18]:
mismo.compare.compared_dashboard(compared, comparers=comparers, weights=weights)

It looks like an odds of 50 seems to separate the pairs between non-matches
and matches.
If I hover over the above chart, I can see that pretty much all the "else" comparisons
are in the low cluster, and all the "exact" comparisons are in the high cluster.

In [19]:
odds_threshold = 50
(scored.odds >= odds_threshold).value_counts()

┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ GreaterEqual(odds, 50) ┃ GreaterEqual(odds, 50)_count ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ boolean                │ int64                        │
├────────────────────────┼──────────────────────────────┤
│ False                  │                        55275 │
│ True                   │                        38121 │
└────────────────────────┴──────────────────────────────┘

Let's be really picky and only take the most likely matches as true matches, and
then perform connected components to label each patent with its inventor:

In [77]:
links = scored.filter(_.odds >= odds_threshold)
links = links.cache()
print(links.count().execute())
labeled = mismo.cluster.connected_components(
    links=links, records=featured, label_as="label_predicted"
)
labeled = labeled.relocate("label_predicted", after="label_true").order_by(
    "label_predicted"
)
print(labeled.count().execute())
labeled

38121
2379


┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ record_id ┃ label_true ┃ label_predicted ┃ name_true            ┃ name                         ┃ name_cleaned         ┃ name_tokens                 ┃ latitude  ┃ longitude ┃ coauthors                        ┃ classes                   ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ uint32    │ uint32     │ int64           │ string               │ string                       │ string               │ array<string>               │ float64   │ float64   │ array<string>                    │ array<string>             │
├───────────┼────────────┼─────────────────┼──────────────────────┼──────────────────────────────┼──────────────────────┼─────────────────────────────┼───────────┼───────────┼──────────────────────────────────┼───────────────────────────┤
│      2909 │     402600 │               0 │ AGILENT TECHNOLOGIES │ * AGILENT TECHNOLOGIES, INC. │ AGILENT TECHNOLOGIES │ ['AGILENT', 'TECHNOLOGIES'] │      NULL │      NULL │ ['DAVID', 'ELECTRONICS', ... +5] │ ['A61B', 'A61N']          │
│      3574 │     569309 │               1 │ AKZO NOBEL           │ * AKZO NOBEL N.V.            │ AKZO NOBEL           │ ['AKZO', 'NOBEL']           │      NULL │      NULL │ ['ALBERT', 'ALBERTO', ... +31]   │ ['B01L', 'C11D', ... +3]  │
│      3575 │     569309 │               2 │ AKZO NOBEL           │ * AKZO NOBEL NV              │ AKZO NOBEL           │ ['AKZO', 'NOBEL']           │      NULL │      NULL │ ['CHRISTINA', 'ERNEST', ... +11] │ ['B01F', 'C09K', ... +2]  │
│    663086 │     656303 │               3 │ ALCATEL              │ Alcatel N. V.                │ ALCATEL              │ ['ALCATEL']                 │ 52.350000 │  4.916667 │ ['ANDRE', 'BANNIZA', ... +37]    │ ['G02B', 'G02F', ... +12] │
│    663238 │     656303 │               3 │ ALCATEL              │ Alcatel N.V.                 │ ALCATEL              │ ['ALCATEL']                 │ 52.500000 │  5.750000 │ ['BOUARD', 'CHIARONI', ... +7]   │ ['H04J', 'H04L', ... +1]  │
│  13627010 │   13640773 │               3 │ HONEYWELL            │ HONEYWELL B.V.               │ HONEYWELL            │ ['HONEYWELL']               │ 52.783333 │  6.900000 │ ['BAARDA', 'DERK', ... +3]       │ ['F23N']                  │
│    663253 │     656303 │               3 │ ALCATEL V            │ ALCATEL N.V., AMSTERDAM, NL  │ ALCATEL AMSTERDAM    │ ['ALCATEL', 'AMSTERDAM']    │ 52.350000 │  4.916667 │ ['0019', '0040', ... +348]       │ ['B01J', 'C01B', ... +30] │
│    663155 │     656303 │               3 │ ALCATEL              │ ALCATEL N.V.                 │ ALCATEL              │ ['ALCATEL']                 │ 52.350000 │  4.916667 │ ['BENZ', 'DELBARE', ... +7]      │ ['G02B', 'G05F', ... +6]  │
│    663243 │     656303 │               3 │ ALCATEL              │ Alcatel N.V.                 │ ALCATEL              │ ['ALCATEL']                 │      NULL │      NULL │ ['ADOLPHI', 'ADRIAN', ... +175]  │ ['C03B', 'G01B', ... +24] │
│    663165 │     656303 │               3 │ ALCATEL              │ ALCATEL N.V.                 │ ALCATEL              │ ['ALCATEL']                 │ 52.350000 │  4.916667 │ ['HELMUT', 'HILDEBRAND', ... +2] │ ['H01B']                  │
│         … │          … │               … │ …                    │ …                            │ …                    │ …                           │         … │         … │ …                                │ …                         │
└───────────┴────────────┴─────────────────┴──────────────────────┴──────────────────────────────┴──────────────────────┴─────────────────────────────┴───────────┴───────────┴─

Now let's evaluate how good our labeling is. Mismo wraps all of the evaluation
metrics from sklearn, so we can use them with Ibis Tables.

In [78]:
labels_true = labeled.select("record_id", label=_.label_true)
labels_pred = labeled.select("record_id", label=_.label_predicted)
print(mismo.cluster.adjusted_rand_score(labels_true, labels_pred))
print(mismo.cluster.homogeneity_score(labels_true, labels_pred))
print(mismo.cluster.completeness_score(labels_true, labels_pred))

/Users/nc/code/mismo/.venv/lib/python3.12/site-packages/sklearn/metrics/cluster/_supervised.py:50: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_pred = type_of_target(labels_pred)
/Users/nc/code/mismo/.venv/lib/python3.12/site-packages/sklearn/metrics/cluster/_supervised.py:50: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_pred = type_of_target(labels_pred)


0.138796241927576
0.9562485849674871
0.5085580861974832


/Users/nc/code/mismo/.venv/lib/python3.12/site-packages/sklearn/metrics/cluster/_supervised.py:50: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_pred = type_of_target(labels_pred)
